In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import logging
import shutil
from omegaconf import OmegaConf
from hydra import initialize, compose
from hydra.core.config_store import ConfigStore
import mlflow
from src.experiments.sner import sner
from pathlib import Path

logging.basicConfig(
    format="%(asctime)s %(levelname)s:%(message)s",
    level=logging.INFO,
    datefmt="%I:%M:%S",
)
logger = logging.getLogger("training")

In [3]:
sentence_length = 110

In [4]:
from tooling.config import Experiment, Transformation

base_experiment_config = Experiment(
    name="Base Config", iterations=1, force=False
)

levels_transformation_config = Transformation(
    description="Levels",
    type="Reduced",
    task="Domain_Level",
    domain_data="Domain_Level",
    activity="Activity",
    stakeholder="Domain_Level",
    system_function="Interaction_Level",
    interaction="Interaction_Level",
    interaction_data="Interaction_Level",
    workspace="Interaction_Level",
    software="System_Level",
    internal_action="System_Level",
    internal_data="System_Level",
)

label_transformation_config = Transformation(
    description="None",
    type="Full",
    task="Task",
    domain_data="Domain_Data",
    activity="Activity",
    stakeholder="Stakeholder",
    system_function="System_Function",
    interaction="Interaction",
    interaction_data="Interaction_Data",
    workspace="Workspace",
    software="System_Level",
    internal_action="System_Level",
    internal_data="System_Level",
)

In [5]:
from classifiers.staged_bert.classifier import get_hint_transformation
import pickle

hint_transformation = get_hint_transformation(
    transformation_cfg=OmegaConf.structured(levels_transformation_config)
)
hint_label2id = hint_transformation["label2id"]
pickle.dump(
    hint_label2id, open("./src/service/models/hint_label2id.pickle", "wb")
)
hint_id2label = {y: x for x, y in hint_label2id.items()}
pickle.dump(
    hint_id2label, open("./src/service/models/hint_id2label.pickle", "wb")
)

transformation = get_hint_transformation(
    transformation_cfg=OmegaConf.structured(label_transformation_config)
)
label2id = transformation["label2id"]
pickle.dump(label2id, open("./src/service/models/label2id.pickle", "wb"))
id2label = {y: x for x, y in label2id.items()}
pickle.dump(id2label, open("./src/service/models/id2label.pickle", "wb"))

07:04:02 INFO:Hint Label2Id: hint_label2id={'0': 0, 'Activity': 1, 'Domain_Level': 2, 'Interaction_Level': 3, 'System_Level': 4}
07:04:02 INFO:Hint Label2Id: hint_label2id={'0': 0, 'Task': 1, 'Domain_Data': 2, 'Activity': 3, 'Stakeholder': 4, 'System_Function': 5, 'Interaction': 6, 'Interaction_Data': 7, 'Workspace': 8, 'System_Level': 9}


## Train BiLSTM First Stage Model

In [6]:
from tooling.observability import get_run_id
from tooling.config import BiLSTMConfig, BiLSTM
from experiments.bilstm import bilstm
from copy import deepcopy

bilstm_experiment_config = deepcopy(base_experiment_config)
bilstm_experiment_config.name = "BiLSTM Train for Staged Application"

bilstm_config = BiLSTM(sentence_length=sentence_length)

bilstm_cfg = OmegaConf.structured(
    BiLSTMConfig(
        bilstm=bilstm_config,
        experiment=bilstm_experiment_config,
        transformation=levels_transformation_config,
    )
)

bilstm(bilstm_cfg)

bilstm_run_id = get_run_id(bilstm_cfg)

print(bilstm_run_id)

bilstm_run = mlflow.get_run(bilstm_run_id)
mlflow.artifacts.download_artifacts(
    f"{bilstm_run.info.artifact_uri}/0_model",
    dst_path=Path("./src/service/models/"),
)
try:
    shutil.rmtree(Path("./src/service/models/bilstm"))
except FileNotFoundError:
    pass
Path("./src/service/models/0_model").rename("./src/service/models/bilstm")

07:04:02 INFO:
bilstm:
  type: BiLSTM
  sentence_length: 110
  batch_size: 32
  number_epochs: 32
  verbose: 1
  weighted_classes: false
  learning_rate: 0.0001
meta:
  mlflow_tracking_uri: https://bockstaller.cc
experiment:
  name: BiLSTM Train for Staged Application
  description: ''
  random_state: 125
  folds: 5
  iterations: 1
  average: macro
  dataset: prolific
  lower_case: false
  force: false
transformation:
  description: Levels
  type: Reduced
  task: Domain_Level
  goals: null
  domain_data: Domain_Level
  activity: Activity
  stakeholder: Domain_Level
  system_function: Interaction_Level
  interaction: Interaction_Level
  interaction_data: Interaction_Level
  workspace: Interaction_Level
  software: System_Level
  internal_action: System_Level
  internal_data: System_Level

07:04:03 WARNING:Experiment was already run, aborting


b96e3ab36442411db9d365dd5ff5d3e5


PosixPath('src/service/models/bilstm')

## Train SNER First Stage Model

In [7]:
from tooling.observability import get_run_id
from tooling.config import SNERConfig, SNER
from experiments.sner import sner
from copy import deepcopy

sner_experiment_config = deepcopy(base_experiment_config)
sner_experiment_config.name = "SNER Train for Staged Application"

sner_config = SNER()

sner_cfg = OmegaConf.structured(
    SNERConfig(
        sner=sner_config,
        experiment=sner_experiment_config,
        transformation=levels_transformation_config,
    )
)

sner(OmegaConf.create(sner_cfg))

sner_run_id = get_run_id(sner_cfg)

print(sner_run_id)

sner_run = mlflow.get_run(sner_run_id)
mlflow.artifacts.download_artifacts(
    f"{sner_run.info.artifact_uri}/0_model.ser.gz",
    dst_path=Path("./src/service/models/"),
)
try:
    Path("./src/service/models/sner.ser.gz").unlink()
except FileNotFoundError:
    pass
Path("./src/service/models/0_model.ser.gz").rename(
    "./src/service/models/sner.ser.gz"
)

07:04:05 INFO:
sner:
  type: SNER
meta:
  mlflow_tracking_uri: https://bockstaller.cc
experiment:
  name: SNER Train for Staged Application
  description: ''
  random_state: 125
  folds: 5
  iterations: 1
  average: macro
  dataset: prolific
  lower_case: false
  force: false
transformation:
  description: Levels
  type: Reduced
  task: Domain_Level
  goals: null
  domain_data: Domain_Level
  activity: Activity
  stakeholder: Domain_Level
  system_function: Interaction_Level
  interaction: Interaction_Level
  interaction_data: Interaction_Level
  workspace: Interaction_Level
  software: System_Level
  internal_action: System_Level
  internal_data: System_Level

07:04:05 WARNING:Experiment was already run, aborting


20f5a21e0e7a484f8f8a248a74f0198b


PosixPath('src/service/models/sner.ser.gz')

## Train BERT First Stage Model

In [8]:
from tooling.observability import get_run_id
from tooling.config import BERTConfig, BERT
from experiments.bert import bert
from copy import deepcopy

bert_1_experiment_config = deepcopy(base_experiment_config)
bert_1_experiment_config.name = "BERT_1 Train for Staged Application"

bert_1_config = BERT(max_len=sentence_length)

bert_1_cfg = OmegaConf.structured(
    BERTConfig(
        bert=bert_1_config,
        experiment=bert_1_experiment_config,
        transformation=levels_transformation_config,
    )
)

bert(OmegaConf.create(bert_1_cfg))

bert_1_run_id = get_run_id(bert_1_cfg)

print(bert_1_run_id)

bert_1_run = mlflow.get_run(bert_1_run_id)
mlflow.artifacts.download_artifacts(
    f"{bert_1_run.info.artifact_uri}/0_model",
    dst_path=Path("./src/service/models/"),
)
try:
    shutil.rmtree(Path("./src/service/models/bert_1"))
except FileNotFoundError:
    pass
Path("./src/service/models/0_model").rename("./src/service/models/bert_1")

07:04:06 INFO:
bert:
  model: bert-base-cased
  type: BERT
  max_len: 110
  train_batch_size: 32
  validation_batch_size: 32
  number_epochs: 5
  learning_rate_bert: 2.0e-05
  learning_rate_classifier: 0.01
  weight_decay: 0.01
  weighted_classes: false
meta:
  mlflow_tracking_uri: https://bockstaller.cc
experiment:
  name: BERT_1 Train for Staged Application
  description: ''
  random_state: 125
  folds: 5
  iterations: 1
  average: macro
  dataset: prolific
  lower_case: false
  force: false
transformation:
  description: Levels
  type: Reduced
  task: Domain_Level
  goals: null
  domain_data: Domain_Level
  activity: Activity
  stakeholder: Domain_Level
  system_function: Interaction_Level
  interaction: Interaction_Level
  interaction_data: Interaction_Level
  workspace: Interaction_Level
  software: System_Level
  internal_action: System_Level
  internal_data: System_Level

07:04:06 WARNING:Experiment was already run, aborting


66d4b382cc4944be93de19d0af2a8826


07:04:19 WARNING:Connection pool is full, discarding connection: ma-mlflow.s3.eu-central-1.amazonaws.com. Connection pool size: 10
07:04:19 WARNING:Connection pool is full, discarding connection: ma-mlflow.s3.eu-central-1.amazonaws.com. Connection pool size: 10
07:04:19 WARNING:Connection pool is full, discarding connection: ma-mlflow.s3.eu-central-1.amazonaws.com. Connection pool size: 10
07:04:19 WARNING:Connection pool is full, discarding connection: ma-mlflow.s3.eu-central-1.amazonaws.com. Connection pool size: 10
07:04:19 WARNING:Connection pool is full, discarding connection: ma-mlflow.s3.eu-central-1.amazonaws.com. Connection pool size: 10
07:04:19 WARNING:Connection pool is full, discarding connection: ma-mlflow.s3.eu-central-1.amazonaws.com. Connection pool size: 10


PosixPath('src/service/models/bert_1')

## Train BERT Second Stage Model for BERT First Stage

In [9]:
from tooling.observability import get_run_id
from tooling.config import DualModelStagedBERTConfig, StagedBERT
from experiments.dual_model_staged_bert import dual_stage_bert
from copy import deepcopy

bert_2_bert_experiment_config = deepcopy(base_experiment_config)
bert_2_bert_experiment_config.name = "BERT_2 Train for Staged Application"

bert_2_bert_config = StagedBERT(max_len=sentence_length)

bert_2_bert_cfg = OmegaConf.structured(
    DualModelStagedBERTConfig(
        bert=bert_2_bert_config,
        experiment=bert_2_bert_experiment_config,
        transformation=label_transformation_config,
        first_model_bert=bert_1_cfg,
    )
)

dual_stage_bert(OmegaConf.create(bert_2_bert_cfg))

bert_2_bert_run_id = get_run_id(bert_2_bert_cfg)

print(bert_2_bert_run_id)

bert_2_bert_run = mlflow.get_run(bert_2_bert_run_id)
mlflow.artifacts.download_artifacts(
    f"{bert_2_bert_run.info.artifact_uri}/0_model",
    dst_path=Path("./src/service/models/"),
)
try:
    shutil.rmtree(Path("./src/service/models/bert_2_bert"))
except FileNotFoundError:
    pass
Path("./src/service/models/0_model").rename("./src/service/models/bert_2_bert")

07:04:20 INFO:
first_model_bert:
  bert:
    model: bert-base-cased
    type: BERT
    max_len: 110
    train_batch_size: 32
    validation_batch_size: 32
    number_epochs: 5
    learning_rate_bert: 2.0e-05
    learning_rate_classifier: 0.01
    weight_decay: 0.01
    weighted_classes: false
  meta:
    mlflow_tracking_uri: https://bockstaller.cc
  experiment:
    name: BERT_1 Train for Staged Application
    description: ''
    random_state: 125
    folds: 5
    iterations: 1
    average: macro
    dataset: prolific
    lower_case: false
    force: false
  transformation:
    description: Levels
    type: Reduced
    task: Domain_Level
    goals: null
    domain_data: Domain_Level
    activity: Activity
    stakeholder: Domain_Level
    system_function: Interaction_Level
    interaction: Interaction_Level
    interaction_data: Interaction_Level
    workspace: Interaction_Level
    software: System_Level
    internal_action: System_Level
    internal_data: System_Level
first_model_bil

d8c07b4055614a4ebdf270f4f30ba61e


07:04:34 WARNING:Connection pool is full, discarding connection: ma-mlflow.s3.eu-central-1.amazonaws.com. Connection pool size: 10
07:04:34 WARNING:Connection pool is full, discarding connection: ma-mlflow.s3.eu-central-1.amazonaws.com. Connection pool size: 10
07:04:34 WARNING:Connection pool is full, discarding connection: ma-mlflow.s3.eu-central-1.amazonaws.com. Connection pool size: 10
07:04:34 WARNING:Connection pool is full, discarding connection: ma-mlflow.s3.eu-central-1.amazonaws.com. Connection pool size: 10
07:04:34 WARNING:Connection pool is full, discarding connection: ma-mlflow.s3.eu-central-1.amazonaws.com. Connection pool size: 10
07:04:34 WARNING:Connection pool is full, discarding connection: ma-mlflow.s3.eu-central-1.amazonaws.com. Connection pool size: 10


PosixPath('src/service/models/bert_2_bert')

## Train BERT Second Stage Model for SNER First Stage

In [10]:
from tooling.observability import get_run_id
from tooling.config import DualModelStagedBERTConfig, StagedBERT
from experiments.dual_model_staged_bert import dual_stage_bert
from copy import deepcopy

bert_2_sner_experiment_config = deepcopy(base_experiment_config)
bert_2_sner_experiment_config.name = "BERT_2 Train for Staged Application"

bert_2_sner_config = StagedBERT(max_len=sentence_length)

bert_2_sner_cfg = OmegaConf.structured(
    DualModelStagedBERTConfig(
        bert=bert_2_sner_config,
        experiment=bert_2_sner_experiment_config,
        transformation=label_transformation_config,
        first_model_sner=sner_cfg,
    )
)

dual_stage_bert(OmegaConf.create(bert_2_sner_cfg))

bert_2_sner_run_id = get_run_id(bert_2_sner_cfg)

print(bert_2_sner_run_id)

bert_2_sner_run = mlflow.get_run(bert_2_sner_run_id)
mlflow.artifacts.download_artifacts(
    f"{bert_2_sner_run.info.artifact_uri}/0_model",
    dst_path=Path("./src/service/models/"),
)
try:
    shutil.rmtree(Path("./src/service/models/bert_2_sner"))
except FileNotFoundError:
    pass

Path("./src/service/models/0_model").rename("./src/service/models/bert_2_sner")

07:04:34 INFO:
first_model_bert: null
first_model_bilstm: null
first_model_sner:
  sner:
    type: SNER
  meta:
    mlflow_tracking_uri: https://bockstaller.cc
  experiment:
    name: SNER Train for Staged Application
    description: ''
    random_state: 125
    folds: 5
    iterations: 1
    average: macro
    dataset: prolific
    lower_case: false
    force: false
  transformation:
    description: Levels
    type: Reduced
    task: Domain_Level
    goals: null
    domain_data: Domain_Level
    activity: Activity
    stakeholder: Domain_Level
    system_function: Interaction_Level
    interaction: Interaction_Level
    interaction_data: Interaction_Level
    workspace: Interaction_Level
    software: System_Level
    internal_action: System_Level
    internal_data: System_Level
bert:
  model: bert-base-cased
  type: BERT
  max_len: 110
  train_batch_size: 32
  validation_batch_size: 32
  number_epochs: 5
  learning_rate_bert: 2.0e-05
  learning_rate_classifier: 0.01
  weight_decay:

b15214708cf7420f90cffcc10b1cc845


07:04:48 WARNING:Connection pool is full, discarding connection: ma-mlflow.s3.eu-central-1.amazonaws.com. Connection pool size: 10
07:04:48 WARNING:Connection pool is full, discarding connection: ma-mlflow.s3.eu-central-1.amazonaws.com. Connection pool size: 10
07:04:48 WARNING:Connection pool is full, discarding connection: ma-mlflow.s3.eu-central-1.amazonaws.com. Connection pool size: 10
07:04:48 WARNING:Connection pool is full, discarding connection: ma-mlflow.s3.eu-central-1.amazonaws.com. Connection pool size: 10
07:04:49 WARNING:Connection pool is full, discarding connection: ma-mlflow.s3.eu-central-1.amazonaws.com. Connection pool size: 10
07:04:49 WARNING:Connection pool is full, discarding connection: ma-mlflow.s3.eu-central-1.amazonaws.com. Connection pool size: 10


PosixPath('src/service/models/bert_2_sner')

## Train BERT Second Stage Model for BILSTM First Stage

In [11]:
from tooling.observability import get_run_id
from tooling.config import DualModelStagedBERTConfig, StagedBERT
from experiments.dual_model_staged_bert import dual_stage_bert
from copy import deepcopy

bert_2_bilstm_experiment_config = deepcopy(base_experiment_config)
bert_2_bilstm_experiment_config.name = "BERT_2 Train for Staged Application"

bert_2_bilstm_config = StagedBERT(max_len=sentence_length)

bert_2_bilstm_cfg = OmegaConf.structured(
    DualModelStagedBERTConfig(
        bert=bert_2_bilstm_config,
        experiment=bert_2_bilstm_experiment_config,
        transformation=label_transformation_config,
        first_model_bilstm=bilstm_cfg,
    )
)

dual_stage_bert(OmegaConf.create(bert_2_bilstm_cfg))

bert_2_bilstm_run_id = get_run_id(bert_2_bilstm_cfg)

print(bert_2_bilstm_run_id)

bert_2_bilstm_run = mlflow.get_run(bert_2_bilstm_run_id)
mlflow.artifacts.download_artifacts(
    f"{bert_2_bilstm_run.info.artifact_uri}/0_model",
    dst_path=Path("./src/service/models/"),
)
try:
    shutil.rmtree(Path("./src/service/models/bert_2_bilstm"))
except FileNotFoundError:
    pass
Path("./src/service/models/0_model").rename(
    "./src/service/models/bert_2_bilstm"
)

07:04:49 INFO:
first_model_bert: null
first_model_bilstm:
  bilstm:
    type: BiLSTM
    sentence_length: 110
    batch_size: 32
    number_epochs: 32
    verbose: 1
    weighted_classes: false
    learning_rate: 0.0001
  meta:
    mlflow_tracking_uri: https://bockstaller.cc
  experiment:
    name: BiLSTM Train for Staged Application
    description: ''
    random_state: 125
    folds: 5
    iterations: 1
    average: macro
    dataset: prolific
    lower_case: false
    force: false
  transformation:
    description: Levels
    type: Reduced
    task: Domain_Level
    goals: null
    domain_data: Domain_Level
    activity: Activity
    stakeholder: Domain_Level
    system_function: Interaction_Level
    interaction: Interaction_Level
    interaction_data: Interaction_Level
    workspace: Interaction_Level
    software: System_Level
    internal_action: System_Level
    internal_data: System_Level
first_model_sner: null
bert:
  model: bert-base-cased
  type: BERT
  max_len: 110
  train

665d4700c3e3456298293e40b3a0f08b


07:05:03 WARNING:Connection pool is full, discarding connection: ma-mlflow.s3.eu-central-1.amazonaws.com. Connection pool size: 10
07:05:03 WARNING:Connection pool is full, discarding connection: ma-mlflow.s3.eu-central-1.amazonaws.com. Connection pool size: 10
07:05:03 WARNING:Connection pool is full, discarding connection: ma-mlflow.s3.eu-central-1.amazonaws.com. Connection pool size: 10
07:05:03 WARNING:Connection pool is full, discarding connection: ma-mlflow.s3.eu-central-1.amazonaws.com. Connection pool size: 10
07:05:03 WARNING:Connection pool is full, discarding connection: ma-mlflow.s3.eu-central-1.amazonaws.com. Connection pool size: 10
07:05:03 WARNING:Connection pool is full, discarding connection: ma-mlflow.s3.eu-central-1.amazonaws.com. Connection pool size: 10


PosixPath('src/service/models/bert_2_bilstm')

## Train BERT E2E Model

In [12]:
from tooling.observability import get_run_id
from tooling.config import BERTConfig, BERT
from experiments.bert import bert
from copy import deepcopy

bert_experiment_config = deepcopy(base_experiment_config)
bert_experiment_config.name = "BERT Train for E2E Application"

bert_config = BERT(max_len=sentence_length)

bert_cfg = OmegaConf.structured(
    BERTConfig(
        bert=bert_config,
        experiment=bert_experiment_config,
        transformation=label_transformation_config,
    )
)

bert(OmegaConf.create(bert_cfg))

bert_run_id = get_run_id(bert_cfg)

print(bert_run_id)

run = mlflow.get_run(bert_run_id)
mlflow.artifacts.download_artifacts(
    f"{run.info.artifact_uri}/0_model", dst_path=Path("./src/service/models/")
)
try:
    shutil.rmtree(Path("./src/service/models/bert"))
except FileNotFoundError:
    pass
Path("./src/service/models/0_model").rename("./src/service/models/bert")

07:05:04 INFO:
bert:
  model: bert-base-cased
  type: BERT
  max_len: 110
  train_batch_size: 32
  validation_batch_size: 32
  number_epochs: 5
  learning_rate_bert: 2.0e-05
  learning_rate_classifier: 0.01
  weight_decay: 0.01
  weighted_classes: false
meta:
  mlflow_tracking_uri: https://bockstaller.cc
experiment:
  name: BERT Train for E2E Application
  description: ''
  random_state: 125
  folds: 5
  iterations: 1
  average: macro
  dataset: prolific
  lower_case: false
  force: false
transformation:
  description: None
  type: Full
  task: Task
  goals: null
  domain_data: Domain_Data
  activity: Activity
  stakeholder: Stakeholder
  system_function: System_Function
  interaction: Interaction
  interaction_data: Interaction_Data
  workspace: Workspace
  software: System_Level
  internal_action: System_Level
  internal_data: System_Level

07:05:04 WARNING:Experiment was already run, aborting


9e609bb46052414f91cea9cb5c32f542


07:05:17 WARNING:Connection pool is full, discarding connection: ma-mlflow.s3.eu-central-1.amazonaws.com. Connection pool size: 10
07:05:17 WARNING:Connection pool is full, discarding connection: ma-mlflow.s3.eu-central-1.amazonaws.com. Connection pool size: 10
07:05:17 WARNING:Connection pool is full, discarding connection: ma-mlflow.s3.eu-central-1.amazonaws.com. Connection pool size: 10
07:05:17 WARNING:Connection pool is full, discarding connection: ma-mlflow.s3.eu-central-1.amazonaws.com. Connection pool size: 10
07:05:18 WARNING:Connection pool is full, discarding connection: ma-mlflow.s3.eu-central-1.amazonaws.com. Connection pool size: 10


PosixPath('src/service/models/bert')